In [1]:
import os
import pandas as pd
import numpy as np
import values, objects
import uuid
import random
from datetime import datetime, timedelta

##### Tables

In [2]:
# CDHDR
cdhdr_cols = ['MANDT', 'OBJECTCLAS', 'OBJECTID', 'CHANGENR', 'USERNAME', 'ERDAT']
cdhdr = pd.DataFrame(columns=cdhdr_cols)

# CDPOS
cdpos_cols = ['MANDT', 'CHANGENR', 'TABNAME', 'FNAME', 'OLDVALUE', 'NEWVALUE']
cdpos = pd.DataFrame(columns=cdpos_cols)

# Materials
mara_cols = ['MANDT', 'MATNR', 'MAKTX']
mara = pd.DataFrame(columns=mara_cols)

# Users
usr01_cols = ['MANDT', 'BNAME', 'UTYPE']
usr01 = pd.DataFrame(columns=usr01_cols)

# Customers
Kna1_cols = ['MANDT', 'KUNNR', 'NAME1', 'LAND1', 'ERDAT', 'ERNAM']
kna1 = pd.DataFrame(columns=Kna1_cols)

# Sales Order header
vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'VSBED', 'FAKSK']
# vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'shp. cond.', 'bil. blk.']
vbak = pd.DataFrame(columns=vbak_cols)

# Sales Order items
vbap_cols = ['MANDT', 'VBELN', 'POSNR', 'KWMENG', 'MATNR', 'NETWR']
vbap =pd.DataFrame(columns=vbap_cols)

# Delivery header
likp_cols = ['MANDT', 'VBELN', 'WADAT', 'LFUHR', 'ERDAT', 'ERNAM']
# likp_cols = ['MANDT', 'id', 'planned. del. date', 'del. date', 'ERDAT', 'ERNAM']
likp = pd.DataFrame(columns=likp_cols)

# Delivery items
lips_cols = ['MANDT', 'VBELN', 'KDAUF', 'POSNR']
# lips_cols = ['MANDT', 'likp.id', 'vbap.vbeln', 'vbap.posnr']
lips = pd.DataFrame(columns=lips_cols)

# Header status
vbuk_cols = ['MANDT', 'VBELN', 'LFSTK', 'FKSTK', 'AEDAT', 'VBTYP']
# vbuk_cols = ['MANDT', 'VBELN', 'del. stat', 'bil. stat', 'changed on', 'sd cat.']
vbuk = pd.DataFrame(columns=vbuk_cols)

# Material movement header
mkpf_cols = ['MANDT', 'MBLNR', 'BLART', 'AEDAT', 'USNAM']
# mkpf_cols = ['MANDT', 'id', 'BLART', 'AEDAT', 'USNAM']
mkpf = pd.DataFrame(columns=mkpf_cols)

# Material movement items
mseg_cols = ['MANDT', 'MBLNR', 'KDAUF', 'KDPOS']
# mseg_cols = ['MANDT', 'mkpf.id', 'vbap.vbeln', 'vbap.posnr']
mseg = pd.DataFrame(columns=mseg_cols)

# Billing header
vbrk_cols = ['MANDT', 'VBELN', 'VBTYP', 'ERNAM', 'ERDAT']
# vbrk_cols = ['MANDT', 'id', 'VBTYP', 'ERNAM', 'ERDAT']
vbrk  =pd.DataFrame(columns=vbrk_cols)

# Billing items
vbrp_cols = ['MANDT', 'VBELN', 'AUBEL', 'POSNR']
# vbrp_cols = ['MANDT', 'id', 'vbak.vbeln', 'vbap.posnr']
vbrp  =pd.DataFrame(columns=vbrp_cols)

##### Helpers

In [3]:
UPTO_YEAR = lambda: timedelta(days=random.randint(1, 365), hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
UPTO_MONTH = lambda: timedelta(days=random.randint(1, 30), hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
UPTO_WEEK = lambda: timedelta(days=random.randint(1, 7), hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
UPTO_DAY = lambda: timedelta(days=0, hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
UPTO_HOUR = lambda: timedelta(days=0, hours=0, minutes=random.randint(0, 59), seconds=random.randint(0, 59))

In [4]:
vbak_len = 100

In [5]:
def generate_random_datetime(start_date, end_date):
    time_delta = end_date - start_date
    random_days = random.randint(0, time_delta.days)
    random_seconds = random.randint(0, 86400)  # 86400 seconds in a day

    random_datetime = start_date + timedelta(days=random_days, seconds=random_seconds)
    return random_datetime

def divide_num_in_normal(num: float, num_parts: int):
    mean = num/2
    std_dev = num/15

    random_numbers = np.random.normal(mean, std_dev, num_parts)
    random_numbers /= sum(random_numbers)

    result = [int(number * num) for number in random_numbers]
    result[-1] += num - sum(result)

    return result

def get_factor_of_transit(name, list_of_conditions):
    "get a value of given a name"
    for condition in list_of_conditions:
        if condition["name"] == name:
            return condition["factor_of_transit"]

In [6]:
# Materials
mara_temp_list = []
MATERIALS = []
for mat in values.material_names:
    rand_id = uuid.uuid4()
    mara_temp_list.append([values.mandt, rand_id, mat['name']])
    MATERIALS.append(objects.Materials(id=rand_id, name=mat['name'], availability=mat['availability']))

mara = pd.concat([mara, pd.DataFrame(mara_temp_list, columns=mara_cols)], ignore_index=True)

In [8]:
# Users
usr01_temp_list = []
USERS = []
for u_name in values.users:
    usr01_temp_list.append([values.mandt, u_name, 'A'])
    USERS.append(objects.Users(u_name, 'A'))
usr01_temp_list.append([values.mandt, 'BATCH_JOB', 'B'])
USERS.append(objects.Users('BATCH_JOB', 'B'))

usr01 = pd.concat([usr01, pd.DataFrame(usr01_temp_list, columns=usr01_cols)], ignore_index=True)

In [8]:
# Customers
class Customer:
    def __init__(self, id, created_at, credit_worthyness, cancellation_rate) -> None:
        self.id = id
        self.created_at = created_at
        self.credit_worthyness = credit_worthyness
        self.cancellation_rate = cancellation_rate

kna1_temp_list = []
companies = []
for company in values.company_names.keys():
    rand_id = uuid.uuid4()
    rand_dt = values.times['customer_create_start'] + UPTO_YEAR()
    rand_usr = random.choice(list(values.users.keys()))
    kna1_temp_list.append([values.mandt, rand_id, company, values.company_names[company][1], rand_dt, rand_usr])
    companies.append(Customer(rand_id, rand_dt, values.company_names[company][0], values.company_names[company][2]))

kna1 = pd.concat([kna1, pd.DataFrame(kna1_temp_list, columns=Kna1_cols)], ignore_index=True)

In [9]:
vbak_temp_list = []
vbap_temp_list = []
likp_temp_list = []
lips_temp_list = []
cdhdr_changes_temp_list = []
cdpos_changes_temp_list = []
vbuk_temp_list = []
mkpf_temp_list = []
mseg_temp_list = []
vbrk_temp_list = []
vbrp_temp_list = []

In [17]:

        
class SalesOrder:
    def __init__(self) -> None:
        global vbap_temp_list
        global likp_temp_list
        global lips_temp_list
        global cdhdr_changes_temp_list
        global cdpos_changes_temp_list
        global mkpf_temp_list
        global mseg_temp_list
        global vbrk_temp_list
        global vbrp_temp_list

        self.mandt = values.mandt
        self.vbeln = uuid.uuid4()
        self.rand_company = random.choice(companies)
        self.erdat = generate_random_datetime(self.rand_company.created_at, values.times['salesorder_create_end'])
        self.ernam = random.choice(list(values.users.keys()))
        self.kunnr = self.rand_company.id
        self.netwr = round(random.uniform(500, 15_000), 2)
        self.vsbed = random.choice(values.shipping_conditions)
        self.faksk = pd.NA

        self.vbaps = []
        self.vbuk = self.vbuk = [values.mandt, self.vbeln, 'SALESORDERCREATED', pd.NA, self.erdat, 'C']

        self.delivery_doc_created_at = None
        self.planned_delivery_dt = None
        self.actual_delivery_dt = None
        self.delivery_released_at = None
        self.goods_shipped_at = None
        self.delivery_confirmed_at = None
        self.invoice_sent_at = None
        self.invoice_cleared_at = None
        self.billing_block_set_at = None
        self.billing_block_removed_at = None
        self.sd_cancelled_at = None
        self.sd_returned_at = None
        self.latest_activity_at = self.erdat

        self.has_billing_block = False
        
        # Create Sales order 
        self.create_vbap_rows(num_items=(random.randint(5, 15)))


    def update_latest_activity_time(self, new_latest_time):
        if self.latest_activity_at < new_latest_time:
            self.latest_activity_at = new_latest_time


    def create_vbap_rows(self, num_items):
        # Activity: Create sales order - item
        vbap_netwrs = divide_num_in_normal(num=self.netwr, num_parts=num_items)
        
        for j in range(num_items):
            rand_material = random.choice(MATERIALS)
            quantity = random.randint(25, 150)

            # add the material to vbap table
            vbap_temp_list.append([values.mandt, self.vbeln, j, quantity, rand_material.id, vbap_netwrs[j]])

            # add vbap to vbak object
            self.vbaps.append(objects.SalesOrderItem(self.vbeln, j, quantity, rand_material.id, vbap_netwrs[j], material=rand_material))

    def generate_delivery_doc(self):
        """
        Activity: Generate delivery document
        affected by:
            - material availability
            - checking and cofirming price and other order details
        affects:
            - cycle time
        """
        self.planned_delivery_dt = self.latest_activity_at + UPTO_WEEK()
        self.actual_delivery_dt = self.planned_delivery_dt if random.uniform(0, 1) > 0.25 else self.planned_delivery_dt + UPTO_WEEK()

        # apply the effect of material availability and user tyoe
        avg_mat_availability = sum([SOi.material.availability for SOi in self.vbaps]) / len([SOi.material.availability for SOi in self.vbaps])
        if avg_mat_availability > 0.6:
            self.delivery_doc_created_at = self.latest_activity_at + UPTO_WEEK()
        else:
            self.delivery_doc_created_at = self.latest_activity_at + (UPTO_WEEK() + UPTO_WEEK())
            self.actual_delivery_dt += UPTO_WEEK() 

        # assign user
        if random.uniform(0, 1) < 0.24: # 30% automation rate
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))
        
        # impact of user type
        if user == 'BATCH_JOB':
            self.delivery_doc_created_at =  self.latest_activity_at + UPTO_HOUR()
        else:
            self.delivery_doc_created_at =  self.latest_activity_at + UPTO_DAY()
        
        self.update_latest_activity_time(new_latest_time=self.delivery_doc_created_at)
        rand_likp_id = uuid.uuid4()
        likp_temp_list.append([values.mandt, rand_likp_id, self.planned_delivery_dt, self.actual_delivery_dt, self.delivery_doc_created_at, user])
        
        self.generate_delivery_doc_item(likp_id=rand_likp_id)
    
    def generate_delivery_doc_item(self, likp_id: uuid.UUID):
        # Activity: Generate delivery document - item
        for vbap_elem in self.vbaps:
            lips_temp_list.append([values.mandt, likp_id, self.vbeln, vbap_elem.posnr])

    def release_delivery(self):
        # Activity: Release delivery
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))
        self.delivery_released_at = self.latest_activity_at + UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=self.delivery_released_at)

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, self.delivery_released_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', pd.NA, 'DELIVERYRELEASED'])
        
        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYRELEASED', pd.NA, self.delivery_released_at, 'C']

    def ship_goods(self):
        # Activity: Ship goods
        rand_mblnr = uuid.uuid4()
        self.goods_shipped_at = self.latest_activity_at + UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=self.goods_shipped_at)

        # assign user
        if random.uniform(0, 1) < 0.3: # 30% automation rate
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))

        mkpf_temp_list.append([values.mandt, rand_mblnr, 'GOODSISSUE', self.goods_shipped_at, user])

        for vbap_elem in self.vbaps:
            mseg_temp_list.append([values.mandt, rand_mblnr, self.vbeln, vbap_elem.posnr])

    def send_invoice(self):
        """
        Activity: Send invoice
        affects
            - cycle time
        properties
            - high automation likelihood
        """

        # assign user
        if random.uniform(0, 1) < 0.3: # 30% automation rate
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))

        # impact of user type
        if user == 'BATCH_JOB':
            self.invoice_sent_at =  self.latest_activity_at + UPTO_HOUR()
        else:
            self.invoice_sent_at =  self.latest_activity_at + UPTO_DAY()
        self.update_latest_activity_time(new_latest_time=self.invoice_sent_at)
        
        rand_vbrk_id = uuid.uuid4()
        vbrk_temp_list.append([values.mandt, rand_vbrk_id, 'INVOICE', user, self.invoice_sent_at])
        
        self.send_invoice_items(vbrk_id=rand_vbrk_id)

    def send_invoice_items(self, vbrk_id: uuid.UUID):
        for vbap_elem in self.vbaps:
            vbrp_temp_list.append([values.mandt, vbrk_id, self.vbeln, vbap_elem.posnr])
    
    def receive_delivery_confirmation(self):
        # Activity: Recieve delivery confirmation
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.delivery_confirmed_at = self.latest_activity_at + UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=self.delivery_confirmed_at)
        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', pd.NA, self.delivery_confirmed_at, 'C']

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, self.delivery_confirmed_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', 'DELIVERYRELEASED', 'DELIVERYCONFIRMED'])

    def clear_invoice(self):
        # Activity: Clear invoice
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.invoice_cleared_at = self.latest_activity_at + UPTO_MONTH()
        self.update_latest_activity_time(new_latest_time=self.invoice_cleared_at)

        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', 'INVOICECLEARED', self.invoice_cleared_at, 'C']

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbeln, rand_changenr, rand_usr, self.invoice_cleared_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'FKSTK', pd.NA, 'INVOICECLEARED'])


In [11]:
class BillingDeviation:
    def __init__(self, vbak: SalesOrder) -> None:
        self.vbak = vbak
        global cdhdr_changes_temp_list
        global cdpos_changes_temp_list
        
    def set_billing_block(self):
        # Activity: Set billing block
        self.vbak.faksk = 'BILLINGBLOCK'

        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.billing_block_set_at = self.vbak.latest_activity_at + UPTO_DAY()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.billing_block_set_at)

        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.billing_block_set_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', pd.NA, 'BILLINGBLOCK'])
        
        self.vbak.has_billing_block = True

    def remove_billing_block(self):
        # Activity: Remove billing block
        self.vbak.faksk = pd.NA

        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.billing_block_removed_at = self.vbak.latest_activity_at + UPTO_WEEK()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.billing_block_removed_at)

        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.billing_block_removed_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', 'BILLINGBLOCK', pd.NA])

In [12]:
class CancelOrReturn:
    def __init__(self, vbak) -> None:
        self.vbak = vbak
    
    def cancel_order(self):
        # Activity: Cancel order
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.sd_cancelled_at = self.vbak.latest_activity_at + UPTO_DAY()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.sd_cancelled_at)

        cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.sd_cancelled_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'h'])
        self.vbak.vbuk[-1] = 'h'

    def return_goods(self):
        # Activity: Return goods
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.sd_returned_at = self.vbak.latest_activity_at + UPTO_MONTH()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.sd_returned_at)

        cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.sd_returned_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'H'])
        self.vbak.vbuk[-1] = 'H'

In [13]:
transition_matrix = np.array([
    #  [gdd., rdel, sbb., shig, rbb., sniv, cano, rdcn, retg, cinv, end.],
       [0.00, 0.90, 0.05, 0.02, 0.01, 0.00, 0.00, 0.00, 0.00, 0.02, 0.00], # generate delivery doc
       [0.00, 0.00, 0.03, 0.90, 0.00, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00], # release delivery
       [0.00, 0.00, 0.00, 0.85, 0.05, 0.00, 0.10, 0.00, 0.00, 0.00, 0.00], # set billing block
       [0.00, 0.00, 0.00, 0.05, 0.05, 0.90, 0.03, 0.03, 0.01, 0.05, 0.00], # ship goods
       [0.00, 0.00, 0.00, 0.00, 0.05, 0.85, 0.05, 0.01, 0.03, 0.01, 0.00], # remove billing block
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.05, 0.90, 0.05, 0.05, 0.00], # send invoice
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # cancel order
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.10, 0.90, 0.05], # receive delivery confirmation
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # return goods
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.95],  # clear invoice
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # end
])
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

for i in range(10_000):
    vbak_obj = SalesOrder()
    billing_dev_obj = BillingDeviation(vbak=vbak_obj)
    cancel_return_dev_obj = CancelOrReturn(vbak=vbak_obj)

    activities = [
        vbak_obj.generate_delivery_doc, 
        vbak_obj.release_delivery, 
        billing_dev_obj.set_billing_block, 
        vbak_obj.ship_goods, 
        billing_dev_obj.remove_billing_block, 
        vbak_obj.send_invoice, 
        cancel_return_dev_obj.cancel_order, 
        vbak_obj.receive_delivery_confirmation, 
        cancel_return_dev_obj.return_goods, 
        vbak_obj.clear_invoice,
        'end'
    ]


    step = 0 if random.uniform(0, 1) < 0.95 else random.choice(list(range(len(activities)-1)))
    while step < len(activities) - 1:
        # take a step
        activities[step]()

        # decide next step
        next_step = np.random.choice(len(activities), p=transition_matrix[step]) # select another steps
        if next_step == 2: # set billing block
            if random.uniform(0, 1) < vbak_obj.rand_company.credit_worthyness: # company is credit worthy
                next_step = np.random.choice(len(activities), p=transition_matrix[step])  # select another step
        elif next_step == 4: # remove billing bloack
            if not (vbak_obj.has_billing_block and random.uniform(0, 1) < 0.90):
                next_step = np.random.choice(len(activities), p=transition_matrix[step]) # select another step
        elif next_step == 6:
            if random.uniform(0, 1) > vbak_obj.rand_company.cancellation_rate:
                next_step = np.random.choice(len(activities), p=transition_matrix[step]) # select another step
        elif next_step == 9:
            if (vbak_obj.planned_delivery_dt is not None and vbak_obj.actual_delivery_dt is not None and vbak_obj.actual_delivery_dt < vbak_obj.planned_delivery_dt + UPTO_WEEK()):
                next_step = np.random.choice(len(activities), p=transition_matrix[step]) # select another step
                
        step = next_step

        # break if end
        if next_step == len(activities) - 1:
                break

    vbuk_temp_list.append(vbak_obj.vbuk)
    vbak_temp_list.append([values.mandt, vbak_obj.vbeln, vbak_obj.erdat, vbak_obj.ernam, vbak_obj.kunnr, vbak_obj.netwr, vbak_obj.vsbed, vbak_obj.faksk])

In [14]:
vbak = pd.concat([vbak, pd.DataFrame(vbak_temp_list, columns=vbak_cols)], ignore_index=True)
vbap = pd.concat([vbap, pd.DataFrame(vbap_temp_list, columns=vbap_cols)], ignore_index=True)
likp = pd.concat([likp, pd.DataFrame(likp_temp_list, columns=likp_cols)], ignore_index=True)
lips = pd.concat([lips, pd.DataFrame(lips_temp_list, columns=lips_cols)], ignore_index=True)
cdhdr = pd.concat([cdhdr, pd.DataFrame(cdhdr_changes_temp_list, columns=cdhdr_cols)], ignore_index=True)
cdpos = pd.concat([cdpos, pd.DataFrame(cdpos_changes_temp_list, columns=cdpos_cols)], ignore_index=True)
vbuk = pd.concat([vbuk, pd.DataFrame(vbuk_temp_list, columns=vbuk_cols)], ignore_index=True)
mkpf = pd.concat([mkpf, pd.DataFrame(mkpf_temp_list, columns=mkpf_cols)], ignore_index=True)
mseg = pd.concat([mseg, pd.DataFrame(mseg_temp_list, columns=mseg_cols)], ignore_index=True)
vbrk = pd.concat([vbrk, pd.DataFrame(vbrk_temp_list, columns=vbrk_cols)], ignore_index=True)
vbrp = pd.concat([vbrp, pd.DataFrame(vbrp_temp_list, columns=vbrp_cols)], ignore_index=True)

##### Save csv

In [15]:
version = str(datetime.now())
os.mkdir(f'data/{version}')
cdhdr.to_csv(f'data/{version}/cdhdr.csv', index=False) 
cdpos.to_csv(f'data/{version}/cdpos.csv', index=False) 
mara.to_csv(f'data/{version}/mara.csv', index=False) 
usr01.to_csv(f'data/{version}/usr01.csv', index=False) 
kna1.to_csv(f'data/{version}/kna1.csv', index=False) 
vbak.to_csv(f'data/{version}/vbak.csv', index=False) 
vbap.to_csv(f'data/{version}/vbap.csv', index=False) 
likp.to_csv(f'data/{version}/likp.csv', index=False) 
lips.to_csv(f'data/{version}/lips.csv', index=False) 
vbuk.to_csv(f'data/{version}/vbuk.csv', index=False) 
mkpf.to_csv(f'data/{version}/mkpf.csv', index=False) 
mseg.to_csv(f'data/{version}/mseg.csv', index=False) 
vbrk .to_csv(f'data/{version}/vbrk.csv', index=False) 
vbrp .to_csv(f'data/{version}/vbrp.csv', index=False) 

In [16]:
vbap

,MANDT,VBELN,POSNR,KWMENG,MATNR,NETWR
0,SC1,13ca8e67-c132-4bfc-81d4-df2a0a99a209,0,87,9004a4f6-1879-468e-a216-a79935d0d6dc,207.00
1,SC1,13ca8e67-c132-4bfc-81d4-df2a0a99a209,1,67,94edae8e-9cae-4fda-9eea-83c9e3476e7e,168.00
2,SC1,13ca8e67-c132-4bfc-81d4-df2a0a99a209,2,82,ff9eecf3-752c-45c2-b468-c6e0b0bd600c,249.00
3,SC1,13ca8e67-c132-4bfc-81d4-df2a0a99a209,3,30,2c1fafce-e4b6-447e-910a-c359a57ffba9,184.00
4,SC1,13ca8e67-c132-4bfc-81d4-df2a0a99a209,4,64,5f2b444e-9c9c-4c48-95e0-c3c6ff98eb77,191.00
...,...,...,...,...,...,...
100508,SC1,340431a3-6280-4015-b649-afe9a16983bc,2,97,e565e42d-1998-4b0b-aa22-d2710a4b5e6a,1189.00
100509,SC1,340431a3-6280-4015-b649-afe9a16983bc,3,138,ff5eed92-66e6-4349-bbff-56f927bc5a73,1111.00
100510,SC1,340431a3-6280-4015-b649-afe9a16983bc,4,110,464ea2f2-4843-4b19-92e0-1179de0d2677,1093.00
100511,SC1,340431a3-6280-4015-b649-afe9a16983bc,5,111,56334a21-1d84-4073-8e70-b9bf434a73e8,999.00
